In [20]:
## Running this file locally might be the best option 


## Libraries

In [220]:
import nltk
import os
import pickle
import re
import shlex
# import stanza
import subprocess
import time

import pandas as pd
import numpy as np

from nltk import word_tokenize
from nltk.tokenize import SyllableTokenizer
from subprocess import Popen, PIPE
from tqdm import tqdm

## Linguistic features import

In [221]:
feat = pd.read_csv('features_all.csv')
feat

,Unnamed: 0,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty,TRSF,shallow
0,0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666...","{'to_TreeH_C': 35, 'as_TreeH_C': 5.83333333333...","{'TokSenM_S': 186.0, 'TokSenS_S': 13.638181696..."
1,1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 18, 'as_TreeH_C': 4.5, 'at_Tree...","{'TokSenM_S': 112.0, 'TokSenS_S': 10.583005244..."
2,2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a...","{'to_TreeH_C': 31, 'as_TreeH_C': 6.2, 'at_Tree...","{'TokSenM_S': 205.0, 'TokSenS_S': 14.317821063..."
3,3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C...","{'to_TreeH_C': 21, 'as_TreeH_C': 7.0, 'at_Tree...","{'TokSenM_S': 84.0, 'TokSenS_S': 9.16515138991..."
4,4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 28, 'as_TreeH_C': 5.6, 'at_Tree...","{'TokSenM_S': 155.0, 'TokSenS_S': 12.449899597..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AAKuW_C': 35.304, '...","

In [108]:
ff1 = eval(feat['WRich05_S'][0].replace("'", "\""))
list(ff1.keys())

['WRich05_S',
 'WRich10_S',
 'WRich15_S',
 'WRich20_S',
 'WClar05_S',
 'WClar10_S',
 'WClar15_S',
 'WClar20_S',
 'WNois05_S',
 'WNois10_S',
 'WNois15_S',
 'WNois20_S',
 'WTopc05_S',
 'WTopc10_S',
 'WTopc15_S',
 'WTopc20_S']

In [109]:
test = np.array(list(ff1.values())).reshape(-1,1).T
test
ff1

{'WRich05_S': 2.158255100250244,
 'WRich10_S': 8.294457910582423,
 'WRich15_S': 14.414342729374766,
 'WRich20_S': 20.872765112668276,
 'WClar05_S': 0.3439428687095642,
 'WClar10_S': 0.3391317486763,
 'WClar15_S': 0.3409104347229004,
 'WClar20_S': 0.39565885398123,
 'WNois05_S': 3.2310979162355444,
 'WNois10_S': 3.6852706518066007,
 'WNois15_S': 3.325639267779427,
 'WNois20_S': 3.561551505654914,
 'WTopc05_S': 5,
 'WTopc10_S': 10,
 'WTopc15_S': 14,
 'WTopc20_S': 18}

In [110]:
df1 = pd.DataFrame(data = test, columns = list(ff1.keys()))
df1

,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.34091,0.395659,3.231098,3.685271,3.325639,3.561552,5.0,10.0,14.0,18.0


In [113]:
ff2 = eval(feat['WRich05_S'][1].replace("'", "\""))
test2 = np.array(list(ff2.values())).reshape(-1,1).T
df2 = pd.DataFrame(data = test2, columns = list(ff2.keys()))
df2

,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,3.612429,4.378085,3.520065,3.713331,5.0,11.0,15.0,19.0


In [119]:
pd.concat([df1,df2])

,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.340910,0.395659,3.231098,3.685271,3.325639,3.561552,5.0,10.0,14.0,18.0
0,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,3.612429,4.378085,3.520065,3.713331,5.0,11.0,15.0,19.0


In [126]:
cols = eval(feat['WRich05_S'][0].replace("'", "\""))
df_tot = pd.DataFrame(columns = list(cols.keys()))
for i in range(len(feat)):
    f = eval(feat['WRich05_S'][i].replace("'", "\""))
    test = np.array(list(f.values())).reshape(-1,1).T
    df = pd.DataFrame(data = test, columns = list(f.keys()))
    df_tot = pd.concat([df_tot, df])
    
df_tot

,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.340910,0.395659,3.231098,3.685271,3.325639,3.561552,5.0,10.0,14.0,18.0
0,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,3.612429,4.378085,3.520065,3.713331,5.0,11.0,15.0,19.0
0,1.748259,7.660348,18.778559,26.862727,0.309756,0.369370,0.372672,0.478896,1.809323,5.014204,4.611439,5.659538,5.0,14.0,22.0,27.0
0,1.779820,7.001251,13.074130,20.780278,0.379472,0.380196,0.398206,0.407678,2.300881,3.933273,3.410577,3.265589,4.0,8.0,12.0,16.0
0,1.957546,6.838163,13.732560,19.437915,0.412763,0.458915,0.458017,0.468887,2.731800,4.383773,3.874885,3.745704,4.0,9.0,13.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2.206126,8.185350,13.203808,20.530431,0.366414,0.341603,0.360324,0.428272,3.173829,3.465366,3.135799,3.639062,5.0,9.0,13.0,17.0
0,2.277161,7.148009,13.077985,21.080375,0.239431,0.217313,0.258199,0.274110,3.068461,2.707114,2.582278,2.616160,4.0,7.0,11.0,15.0
0,2.486698,5.522842,12.253382,20.175007,0.240832,0.314523,0.358045,0.363364,3.013528,3.036050,3.378183,3.266380,4.0,7.0,12.0,16.0
0,1.384804,7.338286,12.370455,20.177510,0.414225,0.485260,0.515215,0.521102,1.590862,5.333898,5.284410,4.590788,3.0,8.0,12.0,16.0


### List of columns

In [127]:
feat.columns

Index(['Unnamed: 0', 'rid', 'qid', 'full', 'pos', 'clicked', 'extract',
       'preprocess', 'WRich05_S', 'phrasal', 'PartsOfSpeech', 'TradFeats',
       'TTRatio', 'Variation', 'PDifficulty', 'TRSF', 'shallow'],
      dtype='object')

### One by one manually

### 1. preprocess

In [132]:
# get the keys of a dict and to be used as columns in this dataframe.
# Initially the dictionally are stings. eval() and .replace() are used to convert the str into a dict
cols_prep = eval(feat['preprocess'][0].replace("'", "\"")) 

# initicate a dataframe with only variable (columns without values)
df_prep = pd.DataFrame(columns = list(cols_prep.keys()))
for i in range(len(feat)): # feat: the oiginal dataset
    f = eval(feat['preprocess'][i].replace("'", "\""))
    val = np.array(list(f.values())).reshape(-1,1).T # reshape the dict value to become the column entries
    df = pd.DataFrame(data = val, columns = list(f.keys()))
    df_prep = pd.concat([df_prep, df])
    
df_prep

,n_token,n_sent
0,31,6
0,28,4
0,41,5
0,28,3
0,31,5
...,...,...
0,27,5
0,22,2
0,22,3
0,19,2


## 2. WRich05_S

In [139]:
# get the columns
cols_wri = eval(feat['WRich05_S'][0].replace("'", "\""))

In [140]:
# initicate a dataframe with only variable (columns without values)
df_wri = pd.DataFrame(columns = list(cols_wri.keys()))

for i in range(len(feat)): # feat: the oiginal dataset
    
    # get the keys of a dict to be used as columns in this dataframe.
    # Initially the dictionally are stings. eval() and .replace() are used to convert the str into a dict
    f = eval(feat['WRich05_S'][i].replace("'", "\""))
    val = np.array(list(f.values())).reshape(-1,1).T # reshape the dict value to become the column entries
    df = pd.DataFrame(data = val, columns = list(f.keys()))
    df_wri = pd.concat([df_wri, df])
    
df_wri


,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.340910,0.395659,3.231098,3.685271,3.325639,3.561552,5.0,10.0,14.0,18.0
0,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,3.612429,4.378085,3.520065,3.713331,5.0,11.0,15.0,19.0
0,1.748259,7.660348,18.778559,26.862727,0.309756,0.369370,0.372672,0.478896,1.809323,5.014204,4.611439,5.659538,5.0,14.0,22.0,27.0
0,1.779820,7.001251,13.074130,20.780278,0.379472,0.380196,0.398206,0.407678,2.300881,3.933273,3.410577,3.265589,4.0,8.0,12.0,16.0
0,1.957546,6.838163,13.732560,19.437915,0.412763,0.458915,0.458017,0.468887,2.731800,4.383773,3.874885,3.745704,4.0,9.0,13.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2.206126,8.185350,13.203808,20.530431,0.366414,0.341603,0.360324,0.428272,3.173829,3.465366,3.135799,3.639062,5.0,9.0,13.0,17.0
0,2.277161,7.148009,13.077985,21.080375,0.239431,0.217313,0.258199,0.274110,3.068461,2.707114,2.582278,2.616160,4.0,7.0,11.0,15.0
0,2.486698,5.522842,12.253382,20.175007,0.240832,0.314523,0.358045,0.363364,3.013528,3.036050,3.378183,3.266380,4.0,7.0,12.0,16.0
0,1.384804,7.338286,12.370455,20.177510,0.414225,0.485260,0.515215,0.521102,1.590862,5.333898,5.284410,4.590788,3.0,8.0,12.0,16.0


### Using Fn

In [149]:
def extract_feat(name):
    cols = eval(feat[name][0].replace("'", "\""))
    df_tot = pd.DataFrame(columns = list(cols.keys()))
    for i in range(len(feat)):
        f = eval(feat[name][i].replace("'", "\""))
        val = np.array(list(f.values())).reshape(-1,1).T
        df = pd.DataFrame(data = val, columns = list(f.keys()))
        df_tot = pd.concat([df_tot, df])
    
    return df_tot

In [180]:
173-16

157

In [144]:
feat.columns

Index(['Unnamed: 0', 'rid', 'qid', 'full', 'pos', 'clicked', 'extract',
       'preprocess', 'WRich05_S', 'phrasal', 'PartsOfSpeech', 'TradFeats',
       'TTRatio', 'Variation', 'PDifficulty', 'TRSF', 'shallow'],
      dtype='object')

### 1. preprocess
-

In [146]:
ft_prep = extract_feat('preprocess')
ft_prep

,n_token,n_sent
0,31,6
0,28,4
0,41,5
0,28,3
0,31,5
...,...,...
0,27,5
0,22,2
0,22,3
0,19,2


### 2. WRich05_S
- 

In [147]:
ft_wri = extract_feat('WRich05_S')
ft_wri

,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,WNois05_S,WNois10_S,WNois15_S,WNois20_S,WTopc05_S,WTopc10_S,WTopc15_S,WTopc20_S
0,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.340910,0.395659,3.231098,3.685271,3.325639,3.561552,5.0,10.0,14.0,18.0
0,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,3.612429,4.378085,3.520065,3.713331,5.0,11.0,15.0,19.0
0,1.748259,7.660348,18.778559,26.862727,0.309756,0.369370,0.372672,0.478896,1.809323,5.014204,4.611439,5.659538,5.0,14.0,22.0,27.0
0,1.779820,7.001251,13.074130,20.780278,0.379472,0.380196,0.398206,0.407678,2.300881,3.933273,3.410577,3.265589,4.0,8.0,12.0,16.0
0,1.957546,6.838163,13.732560,19.437915,0.412763,0.458915,0.458017,0.468887,2.731800,4.383773,3.874885,3.745704,4.0,9.0,13.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2.206126,8.185350,13.203808,20.530431,0.366414,0.341603,0.360324,0.428272,3.173829,3.465366,3.135799,3.639062,5.0,9.0,13.0,17.0
0,2.277161,7.148009,13.077985,21.080375,0.239431,0.217313,0.258199,0.274110,3.068461,2.707114,2.582278,2.616160,4.0,7.0,11.0,15.0
0,2.486698,5.522842,12.253382,20.175007,0.240832,0.314523,0.358045,0.363364,3.013528,3.036050,3.378183,3.266380,4.0,7.0,12.0,16.0
0,1.384804,7.338286,12.370455,20.177510,0.414225,0.485260,0.515215,0.521102,1.590862,5.333898,5.284410,4.590788,3.0,8.0,12.0,16.0


In [162]:
ft_wri.columns.is_unique

True

### 3. phrasal
- 

In [148]:
ft_phr = extract_feat('phrasal')
ft_phr

,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,ra_NoPrP_C,ra_NoAjP_C,ra_NoAvP_C,to_VePhr_C,as_VePhr_C,...,ra_AjPrP_C,ra_AjAvP_C,to_AvPhr_C,as_AvPhr_C,at_AvPhr_C,ra_AvNoP_C,ra_AvVeP_C,ra_AvSuP_C,ra_AvPrP_C,ra_AvAjP_C
0,10.0,1.666667,0.322581,1.000000,10.0,5.000000,5.0,0.0,10.0,1.666667,...,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,14.0,3.500000,0.500000,2.333333,0.0,3.500000,0.0,0.0,6.0,1.500000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,15.0,3.000000,0.365854,1.666667,0.0,2.142857,0.0,15.0,9.0,1.800000,...,0.000000,0.000000,1.0,0.200000,0.024390,0.066667,0.111111,0.0,0.142857,0.0
0,8.0,2.666667,0.285714,1.000000,0.0,4.000000,8.0,0.0,8.0,2.666667,...,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,9.0,1.800000,0.290323,0.642857,1.5,2.250000,4.5,3.0,14.0,2.800000,...,0.500000,0.666667,3.0,0.600000,0.096774,0.333333,0.214286,0.5,0.750000,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,17.0,3.400000,0.629630,5.666667,8.5,8.500000,0.0,0.0,3.0,0.600000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,9.0,4.500000,0.409091,2.250000,9.0,3.000000,9.0,0.0,4.0,2.000000,...,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,12.0,4.000000,0.545455,1.714286,0.0,2.000000,0.0,0.0,7.0,2.333333,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,10.0,5.000000,0.526316,1.666667,0.0,2.000000,0.0,0.0,6.0,3.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [163]:
ft_phr.columns.is_unique

True

### 4. PartsOfSpeech
- 

In [150]:
ft_pos = extract_feat('PartsOfSpeech')
ft_pos

,to_NoTag_C,as_NoTag_C,at_NoTag_C,ra_NoAjT_C,ra_NoVeT_C,ra_NoAvT_C,ra_NoSuT_C,ra_NoCoT_C,to_VeTag_C,as_VeTag_C,...,ra_CoVeT_C,ra_CoAvT_C,ra_CoSuT_C,to_ContW_C,as_ContW_C,at_ContW_C,to_FuncW_C,as_FuncW_C,at_FuncW_C,ra_CoFuW_C
0,8.0,1.333333,0.258065,1.333333,1.000000,0.000000,8.0,4.000000,8.0,1.333333,...,0.250000,0.000000,2.0,23.0,3.833333,0.741935,33.0,5.500000,1.064516,0.696970
0,6.0,1.500000,0.214286,2.000000,2.000000,6.000000,0.0,6.000000,3.0,0.750000,...,0.333333,1.000000,0.0,15.0,3.750000,0.535714,42.0,10.500000,1.500000,0.357143
0,14.0,2.800000,0.341463,4.666667,2.000000,14.000000,0.0,4.666667,7.0,1.400000,...,0.428571,3.000000,0.0,25.0,5.000000,0.609756,53.0,10.600000,1.292683,0.471698
0,11.0,3.666667,0.392857,3.666667,1.833333,11.000000,11.0,3.666667,6.0,2.000000,...,0.500000,3.000000,3.0,21.0,7.000000,0.750000,24.0,8.000000,0.857143,0.875000
0,5.0,1.000000,0.161290,1.250000,0.454545,1.666667,0.0,5.000000,11.0,2.200000,...,0.090909,0.333333,0.0,23.0,4.600000,0.741935,35.0,7.000000,1.129032,0.657143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,0.200000,0.037037,0.500000,0.333333,0.000000,0.0,0.500000,3.0,0.600000,...,0.666667,0.000000,0.0,6.0,1.200000,0.222222,28.0,5.600000,1.037037,0.214286
0,7.0,3.500000,0.318182,3.500000,3.500000,0.000000,0.0,3.500000,2.0,1.000000,...,1.000000,0.000000,0.0,11.0,5.500000,0.500000,19.0,9.500000,0.863636,0.578947
0,6.0,2.000000,0.272727,0.000000,1.200000,0.000000,0.0,6.000000,5.0,1.666667,...,0.200000,0.000000,0.0,13.0,4.333333,0.590909,32.0,10.666667,1.454545,0.406250
0,7.0,3.500000,0.368421,0.000000,1.400000,0.000000,0.0,0.000000,5.0,2.500000,...,0.000000,0.000000,0.0,12.0,6.000000,0.631579,21.0,10.500000,1.105263,0.571429


In [164]:
ft_pos.columns.is_unique

True

### 5. TradFeats
- 

In [151]:
ft_trad = extract_feat('TradFeats')
ft_trad

,FleschG_S,AutoRea_S,ColeLia_S,SmogInd_S,Gunning_S,LinseaW_S
0,12.308871,14.709409,-15.799532,4.358899,2.583333,3.083333
0,9.897143,8.491429,-15.795376,4.000000,2.750000,3.250000
0,7.754341,9.927415,-15.790692,4.000000,4.000000,5.500000
0,15.864286,21.869048,-15.792416,4.123106,6.500000,8.000000
0,7.002194,5.860774,-15.796572,3.605551,2.200000,2.700000
...,...,...,...,...,...,...
0,11.864148,13.623556,-15.798924,3.872983,3.200000,3.700000
0,11.227273,13.630909,-15.792984,3.316625,6.500000,8.000000
0,11.942727,14.132424,-15.795944,3.162278,4.166667,5.666667
0,16.683421,18.385000,-15.794748,3.605551,6.250000,7.750000


In [165]:
ft_trad.columns.is_unique

True

### 6. TTRatio
- 

In [153]:
ft_ttr = extract_feat('TTRatio')
ft_ttr

,SimpTTR_S,CorrTTR_S,BiLoTTR_S,UberTTR_S,MTLDTTR_S
0,0.709677,2.794003,0.900132,27.860299,0.0
0,0.785714,2.939874,0.927627,39.618767,2.0
0,0.780488,3.533809,0.933262,48.464781,3.0
0,0.714286,2.672612,0.899024,26.672072,0.0
0,0.580645,2.286002,0.841696,15.367939,0.0
...,...,...,...,...,...
0,0.629630,2.313407,0.859634,17.351253,0.0
0,0.818182,2.713602,0.935080,41.631634,4.0
0,0.590909,1.959824,0.829801,12.505325,0.0
0,0.894737,2.757764,0.962225,72.169494,4.0


In [166]:
ft_ttr.columns.is_unique

True

### 7. Variation
- 

In [154]:
ft_var = extract_feat('Variation')
ft_var

,SimpNoV_S,SquaNoV_S,CorrNoV_S,SimpVeV_S,SquaVeV_S,CorrVeV_S,SimpAjV_S,SquaAjV_S,CorrAjV_S,SimpAvV_S,SquaAvV_S,CorrAvV_S
0,0.625000,3.125000,1.250000,0.750000,4.500000,1.500000,1.000000,6.000000,1.732051,0.0,0.0,0.000000
0,1.000000,6.000000,1.732051,1.000000,3.000000,1.224745,1.000000,3.000000,1.224745,1.0,1.0,0.707107
0,1.000000,14.000000,2.645751,1.000000,7.000000,1.870829,0.333333,0.333333,0.408248,1.0,1.0,0.707107
0,0.545455,3.272727,1.279204,1.000000,6.000000,1.732051,1.000000,3.000000,1.224745,1.0,1.0,0.707107
0,0.400000,0.800000,0.632456,0.545455,3.272727,1.279204,1.000000,4.000000,1.414214,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.000000,1.000000,0.707107,0.333333,0.333333,0.408248,1.000000,2.000000,1.000000,0.0,0.0,0.000000
0,1.000000,7.000000,1.870829,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.0,0.0,0.000000
0,1.000000,6.000000,1.732051,0.600000,1.800000,0.948683,0.000000,0.000000,0.000000,0.0,0.0,0.000000
0,1.000000,7.000000,1.870829,1.000000,5.000000,1.581139,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [167]:
ft_var.columns.is_unique

True

### 8. PDifficulty
- 

In [155]:
ft_pdi = extract_feat('PDifficulty')
ft_pdi

,to_AAKuW_C,as_AAKuW_C,at_AAKuW_C,to_AAKuL_C,as_AAKuL_C,at_AAKuL_C,to_AABiL_C,as_AABiL_C,at_AABiL_C,to_AABrL_C,as_AABrL_C,at_AABrL_C,to_AACoL_C,as_AACoL_C,at_AACoL_C
0,156.19,26.031667,5.038387,160.51,26.751667,5.177742,66.28,11.046667,2.138065,58.57,9.761667,1.889355,58.57,9.761667,1.889355
0,145.72,36.430000,5.204286,151.98,37.995000,5.427857,80.17,20.042500,2.863214,29.39,7.347500,1.049643,29.39,7.347500,1.049643
0,218.05,43.610000,5.318293,242.98,48.596000,5.926341,96.96,19.392000,2.364878,132.98,26.596000,3.243415,132.98,26.596000,3.243415
0,189.13,63.043333,6.754643,200.24,66.746667,7.151429,86.35,28.783333,3.083929,19.41,6.470000,0.693214,19.41,6.470000,0.693214
0,167.00,33.400000,5.387097,167.00,33.400000,5.387097,101.46,20.292000,3.272903,18.99,3.798000,0.612581,18.99,3.798000,0.612581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,176.52,35.304000,6.537778,200.31,40.062000,7.418889,73.04,14.608000,2.705185,64.69,12.938000,2.395926,64.69,12.938000,2.395926
0,117.23,58.615000,5.328636,117.23,58.615000,5.328636,82.30,41.150000,3.740909,29.66,14.830000,1.348182,29.66,14.830000,1.348182
0,118.95,39.650000,5.406818,118.95,39.650000,5.406818,45.46,15.153333,2.066364,66.03,22.010000,3.001364,66.03,22.010000,3.001364
0,115.60,57.800000,6.084211,115.60,57.800000,6.084211,44.52,22.260000,2.343158,44.36,22.180000,2.334737,44.36,22.180000,2.334737


In [168]:
ft_pdi.columns.is_unique

True

### 9. TRSF
- 

In [156]:
ft_trsf = extract_feat('TRSF')
ft_trsf

,to_TreeH_C,as_TreeH_C,at_TreeH_C,to_FTree_C,as_FTree_C,at_FTree_C
0,35.0,5.833333,1.129032,35.0,5.833333,1.129032
0,18.0,4.500000,0.642857,35.0,8.750000,1.250000
0,31.0,6.200000,0.756098,46.0,9.200000,1.121951
0,21.0,7.000000,0.750000,34.0,11.333333,1.214286
0,28.0,5.600000,0.903226,34.0,6.800000,1.096774
...,...,...,...,...,...,...
0,24.0,4.800000,0.888889,27.0,5.400000,1.000000
0,14.0,7.000000,0.636364,25.0,12.500000,1.136364
0,25.0,8.333333,1.136364,28.0,9.333333,1.272727
0,14.0,7.000000,0.736842,25.0,12.500000,1.315789


In [169]:
ft_trsf.columns.is_unique

True

### 10. shallow
- 

In [158]:
ft_sha = extract_feat('shallow')
ft_sha

,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
0,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
0,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
0,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
0,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...
0,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
0,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
0,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
0,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


In [159]:
ft_sha.columns.is_unique

True

### Concatinate the ft

In [182]:
all_feat = pd.concat([ft_prep, ft_wri, ft_phr, ft_pos, ft_trad, ft_ttr, ft_var, ft_pdi, ft_trsf, ft_sha], axis = 1)
all_feat.reset_index(inplace = True, drop = True)
all_feat

,n_token,n_sent,WRich05_S,WRich10_S,WRich15_S,WRich20_S,WClar05_S,WClar10_S,WClar15_S,WClar20_S,...,as_FTree_C,at_FTree_C,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,31,6,2.158255,8.294458,14.414343,20.872765,0.343943,0.339132,0.340910,0.395659,...,5.833333,1.129032,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
1,28,4,2.363802,8.762073,15.909122,22.452344,0.293427,0.314588,0.304192,0.379115,...,8.750000,1.250000,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
2,41,5,1.748259,7.660348,18.778559,26.862727,0.309756,0.369370,0.372672,0.478896,...,9.200000,1.121951,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
3,28,3,1.779820,7.001251,13.074130,20.780278,0.379472,0.380196,0.398206,0.407678,...,11.333333,1.214286,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
4,31,5,1.957546,6.838163,13.732560,19.437915,0.412763,0.458915,0.458017,0.468887,...,6.800000,1.096774,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,27,5,2.206126,8.185350,13.203808,20.530431,0.366414,0.341603,0.360324,0.428272,...,5.400000,1.000000,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
10288,22,2,2.277161,7.148009,13.077985,21.080375,0.239431,0.217313,0.258199,0.274110,...,12.500000,1.136364,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
10289,22,3,2.486698,5.522842,12.253382,20.175007,0.240832,0.314523,0.358045,0.363364,...,9.333333,1.272727,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
10290,19,2,1.384804,7.338286,12.370455,20.177510,0.414225,0.485260,0.515215,0.521102,...,12.500000,1.315789,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


In [185]:
# Wiki features excluded
all_feat_new = pd.concat([ft_prep, ft_phr, ft_pos, ft_trad, ft_ttr, ft_var, ft_pdi, ft_trsf, ft_sha], axis = 1)
all_feat_new.reset_index(inplace = True, drop = True)
all_feat_new

,n_token,n_sent,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,ra_NoPrP_C,ra_NoAjP_C,ra_NoAvP_C,...,as_FTree_C,at_FTree_C,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,31,6,10.0,1.666667,0.322581,1.000000,10.0,5.000000,5.0,0.0,...,5.833333,1.129032,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
1,28,4,14.0,3.500000,0.500000,2.333333,0.0,3.500000,0.0,0.0,...,8.750000,1.250000,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
2,41,5,15.0,3.000000,0.365854,1.666667,0.0,2.142857,0.0,15.0,...,9.200000,1.121951,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
3,28,3,8.0,2.666667,0.285714,1.000000,0.0,4.000000,8.0,0.0,...,11.333333,1.214286,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
4,31,5,9.0,1.800000,0.290323,0.642857,1.5,2.250000,4.5,3.0,...,6.800000,1.096774,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,27,5,17.0,3.400000,0.629630,5.666667,8.5,8.500000,0.0,0.0,...,5.400000,1.000000,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
10288,22,2,9.0,4.500000,0.409091,2.250000,9.0,3.000000,9.0,0.0,...,12.500000,1.136364,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
10289,22,3,12.0,4.000000,0.545455,1.714286,0.0,2.000000,0.0,0.0,...,9.333333,1.272727,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
10290,19,2,10.0,5.000000,0.526316,1.666667,0.0,2.000000,0.0,0.0,...,12.500000,1.315789,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


### Add results and class column to ft

In [186]:
res_cl = feat[['full', 'clicked']]
res_cl

,full,clicked
0,Incoming Webhooks — Mattermost 5.11 documentat...,1
1,“Hello” Is Facebook's New Android-Only Social ...,0
2,Walkthrough | HTTP Event CollectorHEC tokens a...,0
3,College Scholarships - Scholarships.comMost co...,0
4,How to start sending email — Mailgun API docum...,0
...,...,...
10287,"Science, Technology, Engineering, and Math, in...",1
10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,0
10289,Stem Definition & Meaning - Merriam-WebsterKid...,0
10290,Driven By StemStem Holdings has positioned its...,0


In [189]:
all_feat_final_new = pd.concat([res_cl, all_feat_new], axis = 1)
all_feat_final_new

,full,clicked,n_token,n_sent,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,ra_NoPrP_C,...,as_FTree_C,at_FTree_C,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,Incoming Webhooks — Mattermost 5.11 documentat...,1,31,6,10.0,1.666667,0.322581,1.000000,10.0,5.000000,...,5.833333,1.129032,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
1,“Hello” Is Facebook's New Android-Only Social ...,0,28,4,14.0,3.500000,0.500000,2.333333,0.0,3.500000,...,8.750000,1.250000,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
2,Walkthrough | HTTP Event CollectorHEC tokens a...,0,41,5,15.0,3.000000,0.365854,1.666667,0.0,2.142857,...,9.200000,1.121951,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
3,College Scholarships - Scholarships.comMost co...,0,28,3,8.0,2.666667,0.285714,1.000000,0.0,4.000000,...,11.333333,1.214286,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
4,How to start sending email — Mailgun API docum...,0,31,5,9.0,1.800000,0.290323,0.642857,1.5,2.250000,...,6.800000,1.096774,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",1,27,5,17.0,3.400000,0.629630,5.666667,8.5,8.500000,...,5.400000,1.000000,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,0,22,2,9.0,4.500000,0.409091,2.250000,9.0,3.000000,...,12.500000,1.136364,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
10289,Stem Definition & Meaning - Merriam-WebsterKid...,0,22,3,12.0,4.000000,0.545455,1.714286,0.0,2.000000,...,9.333333,1.272727,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
10290,Driven By StemStem Holdings has positioned its...,0,19,2,10.0,5.000000,0.526316,1.666667,0.0,2.000000,...,12.500000,1.315789,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


In [190]:
### Export the dataframe
all_feat_final_new.to_csv('all_feat_final_new.csv')


## Title LingFeat 

### Load `Title` data

In [195]:
title_feat_data = pd.read_csv('title_features.csv')
title_feat_data

,Unnamed: 0,title,snippet,full,pos,clicked,extract,preprocess,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty,TRSF,shallow
0,0,Incoming Webhooks — Mattermost 5.11 documentation,Use incoming webhooks to post messages to Matt...,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666...","{'to_TreeH_C': 35, 'as_TreeH_C': 5.83333333333...","{'TokSenM_S': 186.0, 'TokSenS_S': 13.638181696..."
1,1,“Hello” Is Facebook's New Android-Only Social ...,"Apr 22, 2015 ... Today, Hello is rolling out f...",“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 18, 'as_TreeH_C': 4.5, 'at_Tree...","{'TokSenM_S': 112.0, 'TokSenS_S': 10.583005244..."
2,2,Walkthrough | HTTP Event Collector,HEC tokens are sent in the headers of incoming...,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a...","{'to_TreeH_C': 31, 'as_TreeH_C': 6.2, 'at_Tree...","{'TokSenM_S': 205.0, 'TokSenS_S': 14.317821063..."
3,3,College Scholarships - Scholarships.com,Most college scholarships target incoming fres...,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C...","{'to_TreeH_C': 21, 'as_TreeH_C': 7.0, 'at_Tree...","{'TokSenM_S': 84.0, 'TokSenS_S': 9.16515138991..."
4,4,How to start sending email — Mailgun API docum...,"It's up to you, whatever you find easier is fi...",How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 28, 'as_TreeH_C': 5.6, 'at_Tree...","{'TokSenM_S': 155.0, 'TokSenS_S': 12.449899597..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,10287,"Science, Technology, Engineering, and Math, in...",Department Offices that Support STEM Open ED F...,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AA

In [226]:
title_feat_data['PartsOfSpeech']

0        {'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...
1        {'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...
2        {'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...
3        {'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...
4        {'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...
                               ...                        
10287    {'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...
10288    {'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...
10289    {'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT...
10290    {'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...
10291    {'to_NoTag_C': 0.0, 'as_NoTag_C': 0.0, 'at_NoT...
Name: PartsOfSpeech, Length: 10292, dtype: object

In [227]:
feat['PartsOfSpeech']

0        {'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...
1        {'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...
2        {'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...
3        {'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...
4        {'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...
                               ...                        
10287    {'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...
10288    {'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...
10289    {'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT...
10290    {'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...
10291    {'to_NoTag_C': 0.0, 'as_NoTag_C': 0.0, 'at_NoT...
Name: PartsOfSpeech, Length: 10292, dtype: object

In [197]:
title_feat_data.columns

Index(['Unnamed: 0', 'title', 'snippet', 'full', 'pos', 'clicked', 'extract',
       'preprocess', 'phrasal', 'PartsOfSpeech', 'TradFeats', 'TTRatio',
       'Variation', 'PDifficulty', 'TRSF', 'shallow'],
      dtype='object')

### Function to transform dicti into cols

In [198]:
# def extract_feat(name):
#     cols = eval(feat[name][0].replace("'", "\""))
#     df_tot = pd.DataFrame(columns = list(cols.keys()))
#     for i in range(len(feat)):
#         f = eval(feat[name][i].replace("'", "\""))
#         val = np.array(list(f.values())).reshape(-1,1).T
#         df = pd.DataFrame(data = val, columns = list(f.keys()))
#         df_tot = pd.concat([df_tot, df])
    
#     return df_tot

In [204]:
def extract_title_feat(name):
    cols = eval(title_feat_data[name][0].replace("'", "\""))
    df_tot = pd.DataFrame(columns = list(cols.keys()))
    for i in range(len(title_feat_data)):
        f = eval(title_feat_data[name][i].replace("'", "\""))
        val = np.array(list(f.values())).reshape(-1,1).T
        df = pd.DataFrame(data = val, columns = list(f.keys()))
        df_tot = pd.concat([df_tot, df])
    
    return df_tot

### 1. preprocess_tit

In [205]:
ft_prep_tit = extract_title_feat('preprocess')
ft_prep_tit

,n_token,n_sent
0,31,6
0,28,4
0,41,5
0,28,3
0,31,5
...,...,...
0,27,5
0,22,2
0,22,3
0,19,2


In [203]:
# ft_wri_tit = extract_title_feat('WRich05_S')
# ft_wri_tit

### 2. phrasal_tit

In [207]:
ft_phr_tit = extract_title_feat('phrasal')
ft_phr_tit

,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,ra_NoPrP_C,ra_NoAjP_C,ra_NoAvP_C,to_VePhr_C,as_VePhr_C,...,ra_AjPrP_C,ra_AjAvP_C,to_AvPhr_C,as_AvPhr_C,at_AvPhr_C,ra_AvNoP_C,ra_AvVeP_C,ra_AvSuP_C,ra_AvPrP_C,ra_AvAjP_C
0,10.0,1.666667,0.322581,1.000000,10.0,5.000000,5.0,0.0,10.0,1.666667,...,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,14.0,3.500000,0.500000,2.333333,0.0,3.500000,0.0,0.0,6.0,1.500000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,15.0,3.000000,0.365854,1.666667,0.0,2.142857,0.0,15.0,9.0,1.800000,...,0.000000,0.000000,1.0,0.200000,0.024390,0.066667,0.111111,0.0,0.142857,0.0
0,8.0,2.666667,0.285714,1.000000,0.0,4.000000,8.0,0.0,8.0,2.666667,...,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,9.0,1.800000,0.290323,0.642857,1.5,2.250000,4.5,3.0,14.0,2.800000,...,0.500000,0.666667,3.0,0.600000,0.096774,0.333333,0.214286,0.5,0.750000,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,17.0,3.400000,0.629630,5.666667,8.5,8.500000,0.0,0.0,3.0,0.600000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,9.0,4.500000,0.409091,2.250000,9.0,3.000000,9.0,0.0,4.0,2.000000,...,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,12.0,4.000000,0.545455,1.714286,0.0,2.000000,0.0,0.0,7.0,2.333333,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
0,10.0,5.000000,0.526316,1.666667,0.0,2.000000,0.0,0.0,6.0,3.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


### 3. PartsOfSpeech_tit

In [208]:
ft_pos_tit = extract_title_feat('PartsOfSpeech')
ft_pos_tit

,to_NoTag_C,as_NoTag_C,at_NoTag_C,ra_NoAjT_C,ra_NoVeT_C,ra_NoAvT_C,ra_NoSuT_C,ra_NoCoT_C,to_VeTag_C,as_VeTag_C,...,ra_CoVeT_C,ra_CoAvT_C,ra_CoSuT_C,to_ContW_C,as_ContW_C,at_ContW_C,to_FuncW_C,as_FuncW_C,at_FuncW_C,ra_CoFuW_C
0,8.0,1.333333,0.258065,1.333333,1.000000,0.000000,8.0,4.000000,8.0,1.333333,...,0.250000,0.000000,2.0,23.0,3.833333,0.741935,33.0,5.500000,1.064516,0.696970
0,6.0,1.500000,0.214286,2.000000,2.000000,6.000000,0.0,6.000000,3.0,0.750000,...,0.333333,1.000000,0.0,15.0,3.750000,0.535714,42.0,10.500000,1.500000,0.357143
0,14.0,2.800000,0.341463,4.666667,2.000000,14.000000,0.0,4.666667,7.0,1.400000,...,0.428571,3.000000,0.0,25.0,5.000000,0.609756,53.0,10.600000,1.292683,0.471698
0,11.0,3.666667,0.392857,3.666667,1.833333,11.000000,11.0,3.666667,6.0,2.000000,...,0.500000,3.000000,3.0,21.0,7.000000,0.750000,24.0,8.000000,0.857143,0.875000
0,5.0,1.000000,0.161290,1.250000,0.454545,1.666667,0.0,5.000000,11.0,2.200000,...,0.090909,0.333333,0.0,23.0,4.600000,0.741935,35.0,7.000000,1.129032,0.657143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,0.200000,0.037037,0.500000,0.333333,0.000000,0.0,0.500000,3.0,0.600000,...,0.666667,0.000000,0.0,6.0,1.200000,0.222222,28.0,5.600000,1.037037,0.214286
0,7.0,3.500000,0.318182,3.500000,3.500000,0.000000,0.0,3.500000,2.0,1.000000,...,1.000000,0.000000,0.0,11.0,5.500000,0.500000,19.0,9.500000,0.863636,0.578947
0,6.0,2.000000,0.272727,0.000000,1.200000,0.000000,0.0,6.000000,5.0,1.666667,...,0.200000,0.000000,0.0,13.0,4.333333,0.590909,32.0,10.666667,1.454545,0.406250
0,7.0,3.500000,0.368421,0.000000,1.400000,0.000000,0.0,0.000000,5.0,2.500000,...,0.000000,0.000000,0.0,12.0,6.000000,0.631579,21.0,10.500000,1.105263,0.571429


### 4. TradFeats_tit

In [209]:
ft_trad_tit = extract_title_feat('TradFeats')
ft_trad_tit

,FleschG_S,AutoRea_S,ColeLia_S,SmogInd_S,Gunning_S,LinseaW_S
0,12.308871,14.709409,-15.799532,4.358899,2.583333,3.083333
0,9.897143,8.491429,-15.795376,4.000000,2.750000,3.250000
0,7.754341,9.927415,-15.790692,4.000000,4.000000,5.500000
0,15.864286,21.869048,-15.792416,4.123106,6.500000,8.000000
0,7.002194,5.860774,-15.796572,3.605551,2.200000,2.700000
...,...,...,...,...,...,...
0,11.864148,13.623556,-15.798924,3.872983,3.200000,3.700000
0,11.227273,13.630909,-15.792984,3.316625,6.500000,8.000000
0,11.942727,14.132424,-15.795944,3.162278,4.166667,5.666667
0,16.683421,18.385000,-15.794748,3.605551,6.250000,7.750000


### TTRatio_tit

In [211]:
ft_ttr_tit = extract_title_feat('TTRatio')
ft_ttr_tit

,SimpTTR_S,CorrTTR_S,BiLoTTR_S,UberTTR_S,MTLDTTR_S
0,0.709677,2.794003,0.900132,27.860299,0.0
0,0.785714,2.939874,0.927627,39.618767,2.0
0,0.780488,3.533809,0.933262,48.464781,3.0
0,0.714286,2.672612,0.899024,26.672072,0.0
0,0.580645,2.286002,0.841696,15.367939,0.0
...,...,...,...,...,...
0,0.629630,2.313407,0.859634,17.351253,0.0
0,0.818182,2.713602,0.935080,41.631634,4.0
0,0.590909,1.959824,0.829801,12.505325,0.0
0,0.894737,2.757764,0.962225,72.169494,4.0


### Load `snippet` data

In [215]:
snip_feat_data = pd.read_csv('snippet_features.csv')
snip_feat_data

,Unnamed: 0,title,snippet,full,pos,clicked,extract,preprocess,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty,TRSF,shallow
0,0,Incoming Webhooks — Mattermost 5.11 documentation,Use incoming webhooks to post messages to Matt...,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666...","{'to_TreeH_C': 35, 'as_TreeH_C': 5.83333333333...","{'TokSenM_S': 186.0, 'TokSenS_S': 13.638181696..."
1,1,“Hello” Is Facebook's New Android-Only Social ...,"Apr 22, 2015 ... Today, Hello is rolling out f...",“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 18, 'as_TreeH_C': 4.5, 'at_Tree...","{'TokSenM_S': 112.0, 'TokSenS_S': 10.583005244..."
2,2,Walkthrough | HTTP Event Collector,HEC tokens are sent in the headers of incoming...,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a...","{'to_TreeH_C': 31, 'as_TreeH_C': 6.2, 'at_Tree...","{'TokSenM_S': 205.0, 'TokSenS_S': 14.317821063..."
3,3,College Scholarships - Scholarships.com,Most college scholarships target incoming fres...,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C...","{'to_TreeH_C': 21, 'as_TreeH_C': 7.0, 'at_Tree...","{'TokSenM_S': 84.0, 'TokSenS_S': 9.16515138991..."
4,4,How to start sending email — Mailgun API docum...,"It's up to you, whatever you find easier is fi...",How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 28, 'as_TreeH_C': 5.6, 'at_Tree...","{'TokSenM_S': 155.0, 'TokSenS_S': 12.449899597..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,10287,"Science, Technology, Engineering, and Math, in...",Department Offices that Support STEM Open ED F...,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AA

In [217]:
def extract_snip_feat(name):
    cols = eval(snip_feat_data[name][0].replace("'", "\""))
    df_tot = pd.DataFrame(columns = list(cols.keys()))
    for i in range(len(snip_feat_data)):
        f = eval(snip_feat_data[name][i].replace("'", "\""))
        val = np.array(list(f.values())).reshape(-1,1).T
        df = pd.DataFrame(data = val, columns = list(f.keys()))
        df_tot = pd.concat([df_tot, df])
    
    return df_tot

In [218]:
ft_ttr_snip = extract_snip_feat('TTRatio')
ft_ttr_snip

,SimpTTR_S,CorrTTR_S,BiLoTTR_S,UberTTR_S,MTLDTTR_S
0,0.709677,2.794003,0.900132,27.860299,0.0
0,0.785714,2.939874,0.927627,39.618767,2.0
0,0.780488,3.533809,0.933262,48.464781,3.0
0,0.714286,2.672612,0.899024,26.672072,0.0
0,0.580645,2.286002,0.841696,15.367939,0.0
...,...,...,...,...,...
0,0.629630,2.313407,0.859634,17.351253,0.0
0,0.818182,2.713602,0.935080,41.631634,4.0
0,0.590909,1.959824,0.829801,12.505325,0.0
0,0.894737,2.757764,0.962225,72.169494,4.0


In [219]:
ft_ttr

,SimpTTR_S,CorrTTR_S,BiLoTTR_S,UberTTR_S,MTLDTTR_S
0,0.709677,2.794003,0.900132,27.860299,0.0
0,0.785714,2.939874,0.927627,39.618767,2.0
0,0.780488,3.533809,0.933262,48.464781,3.0
0,0.714286,2.672612,0.899024,26.672072,0.0
0,0.580645,2.286002,0.841696,15.367939,0.0
...,...,...,...,...,...
0,0.629630,2.313407,0.859634,17.351253,0.0
0,0.818182,2.713602,0.935080,41.631634,4.0
0,0.590909,1.959824,0.829801,12.505325,0.0
0,0.894737,2.757764,0.962225,72.169494,4.0
